# 데이터 전처리

In [114]:
from datetime import datetime
from tqdm import tqdm
import os

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [ ]:
df_raw = pd.read_csv('../data/csv/2019-Nov_1.csv',index_col=0)

In [ ]:
df_raw.head()

## 상위 누적 구매액 10% 유저 추출

In [5]:
# 상품 구매 상위 10% 유저 구하기 위해 누적 금액 계산

df_purchase_amount = df_raw.query('event_type == "purchase"').reset_index()
# del df_purchase_amount['index']
df_purchase_amount.head()

,index,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,category_code_1,category_code_2,category_code_3,category_code_4
0,107,2019-11-01 00:00:41 UTC,purchase,13200605,2053013557192163841,furniture.bedroom.bed,NaN,566.30,559368633,d6034fa2-41fb-4ac0-9051-55ea9fc9147a,furniture,bedroom,bed,NaN
1,168,2019-11-01 00:01:04 UTC,purchase,1005161,2053013555631882655,electronics.smartphone,xiaomi,211.92,513351129,e6b7ce9b-1938-4e20-976c-8b4163aea11d,electronics,NaN,smartphone,NaN
2,707,2019-11-01 00:04:51 UTC,purchase,1004856,2053013555631882655,electronics.smartphone,samsung,128.42,562958505,0f039697-fedc-40fa-8830-39c1a024351d,electronics,NaN,smartphone,NaN
3,811,2019-11-01 00:05:34 UTC,purchase,26401669,2053013563651392361,NaN,lucente,109.66,541854711,c41c44d5-ef9b-41b9-9cd6-8d96dda6e927,NaN,NaN,NaN,NaN
4,939,2019-11-01 00:06:33 UTC,purchase,1801881,2053013554415534427,electronics.video.tv,samsung,488.80,557746614,4d76d6d3-fff5-4880-8327-e9e57b618e0e,electronics,video,tv,NaN


In [6]:
dic_acc_price = {}
for idx in tqdm(range(len(df_purchase_amount))):
    dic_acc_price['{}'.format(df_purchase_amount['user_id'][idx])] = 0
    
for idx in tqdm(range(len(df_purchase_amount))):
    dic_acc_price['{}'.format(df_purchase_amount['user_id'][idx])] += df_purchase_amount['price'][idx]

100%|█████████████████████████████████| 916939/916939 [00:06<00:00, 142812.93it/s]


In [7]:
# 개별 유저의 제품 누적 구매액

df_user = pd.DataFrame()

In [8]:
df_user['user_id'] = dic_acc_price.keys()
df_user['acc_purchase'] = dic_acc_price.values()

In [9]:
df_user = df_user.sort_values(by =['acc_purchase'],ascending=False)

In [10]:
n = 1*(len(df_user)//10)
df_user_top10 = df_user.iloc[:int(n)]

In [11]:
df_user_top10.head()

,user_id,acc_purchase
230571,518514099,203986.07
2523,512386086,161682.49
5589,564068124,131929.20
970,515715331,117130.12
187568,521230795,111072.35


In [12]:
lst_user_top10 = df_user_top10['user_id'].to_list()
dic_user_top10 = {}
for idx in lst_user_top10:
    dic_user_top10[f'{idx}'] = 1

In [16]:
df_raw['user_top10'] = 0
for idx in tqdm(range(len(df_raw))):
    try:
        df_raw['user_top10'][idx] = dic_user_top10['{}'.format(df_raw['user_id'][idx])]
    except:
        continue

100%|█████████████████████████████| 67501979/67501979 [2:45:19<00:00, 6804.80it/s]


In [18]:
df_raw['user_top10'].value_counts()

0    63991603
1     3510376
Name: user_top10, dtype: int64

In [33]:
df_top10 = df_raw.query('user_top10 == 1').reset_index()
del df_top10['index']
df_top10.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,category_code_1,category_code_2,category_code_3,category_code_4,user_top10
0,2019-11-01 00:00:19 UTC,view,26100002,2053013553887052089,NaN,sulu,51.48,520956879,88e782d0-cb8f-4b25-b09c-6dd9bce06832,NaN,NaN,NaN,NaN,1
1,2019-11-01 00:00:35 UTC,view,2701305,2053013563911439225,appliances.kitchen.refrigerators,samsung,499.94,524356542,153f9818-4d32-4e8b-ba9f-f355094e8ae4,appliances,kitchen,refrigerators,NaN,1
2,2019-11-01 00:01:46 UTC,view,39700019,2084962304016777941,apparel.costume,NaN,136.43,516426931,b36a7084-888d-4e37-bad4-cec9cfb1fbce,apparel,NaN,costume,NaN,1
3,2019-11-01 00:02:02 UTC,view,1004873,2053013555631882655,electronics.smartphone,samsung,362.29,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10,electronics,NaN,smartphone,NaN,1
4,2019-11-01 00:02:36 UTC,view,5100816,2053013553375346967,NaN,xiaomi,29.58,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10,NaN,NaN,NaN,NaN,1


In [34]:
df_top10.describe()

,product_id,category_id,price,user_id,user_top10
count,3.510376e+06,3.510376e+06,3.510376e+06,3.510376e+06,3510376.0
mean,9.618458e+06,2.056471e+18,4.787635e+02,5.391820e+08,1.0
std,1.643845e+07,1.756936e+16,4.660808e+02,2.173473e+07,0.0
min,1.000978e+06,2.053014e+18,0.000000e+00,3.047076e+08,1.0
25%,1.005105e+06,2.053014e+18,1.275500e+02,5.165548e+08,1.0
50%,1.801929e+06,2.053014e+18,2.977400e+02,5.378731e+08,1.0
75%,1.271256e+07,2.053014e+18,7.457000e+02,5.603779e+08,1.0
max,1.000286e+08,2.187708e+18,2.574070e+03,5.799627e+08,1.0


In [35]:
df_top10['event_type'].value_counts()

view        2750853
cart         476078
purchase     283445
Name: event_type, dtype: int64

## user_session == 1 값 제거
- user_session == 1인 경우, 유의미한 행동 패턴 분석이 어렵다 판단하여 1값 제거

In [40]:
# user_session 중복값 세기(v_c : value_count)

v_c = df_top10['user_session'].value_counts()
v_c

68b52b9a-97c8-4525-aba2-604ede028da8    319
c491177d-d680-4104-8b08-bc6952e910d9    306
31a89207-dd7b-4bbe-8678-6a3be879fe8c    294
c200e51b-7c35-4efa-9956-a5e32e3dfdfc    279
fb8a1633-dcbb-4f98-b00c-a0dd9b61c808    279
                                       ... 
42a02b65-01b2-4e71-ab40-caf60573d019      1
b94c0ba1-8525-428b-b1f4-0b53ac3d6837      1
a715dbce-d0f2-4935-9770-0c79d2f60693      1
d1de3ccb-1c26-4115-bebf-14b06df506e4      1
cc3328ce-e22a-49ba-b92a-3e143fb4b3d6      1
Name: user_session, Length: 582552, dtype: int64

In [41]:
lst_v_c_idx = v_c.index.to_list()
lst_v_c = v_c.to_list()

In [42]:
dic = {}
for idx, value in zip(lst_v_c_idx, lst_v_c):
    dic[idx] = value

In [43]:
# try ~ except : Error idx 확인

lst_user_session_c = []
df_top10['user_session_c'] = 0
for idx in tqdm(range(len(df_top10))):
    try:
        df_top10['user_session_c'][idx] = dic[df_top10['user_session'][idx]]
    except:
        lst_user_session_c.append(idx)

100%|████████████████████████████████| 3510376/3510376 [2:40:54<00:00, 363.62it/s]


In [45]:
# user_session nan 값 또한 제거를 위해 1로 대입.
for idx in tqdm(lst_user_session_c):
    df_top10['user_session_c'][idx] = 0

0it [00:00, ?it/s]


In [47]:
df = df_top10.query('user_session_c != 1')
df = df.query('user_session_c != 0')

## feature 생성
### 카테고리 분류 나누기 : 대분류, 중분류, 소분류

In [50]:
df.to_csv('0130.csv',encoding='cp949')

In [49]:
# category_code 1: 대분류, 2 : 중분류, 3: 소분류, 4: 소소분류
# 카테고리 분류 결과 더 분류 가능하다 판단해 4까지 진행

df_top10['category_code_1'] = ''
df_top10['category_code_2'] = ''
df_top10['category_code_3'] = ''
df_top10['category_code_4'] = ''

In [51]:
for idx in tqdm(range(len(df_top10))):
    try:
        separate = df_top10['category_code'][idx].split('.')
        if len(separate) == 2:
            df_top10['category_code_1'][idx] = separate[0]
            df_top10['category_code_3'][idx] = separate[1]
        elif len(separate) == 3:
            df_top10['category_code_1'][idx] = separate[0]
            df_top10['category_code_2'][idx] = separate[1]
            df_top10['category_code_3'][idx] = separate[2]
        elif len(separate) == 4:
            df_top10['category_code_1'][idx] = separate[0]
            df_top10['category_code_2'][idx] = separate[1]
            df_top10['category_code_3'][idx] = separate[2]
            df_top10['category_code_4'][idx] = separate[3]
        else:
            print(idx,df_top10['category_code'])
    except:
        continue

100%|████████████████████████████████| 3510376/3510376 [5:47:06<00:00, 168.55it/s]


In [16]:
df_top10 = pd.read_csv('0131.csv',encoding='cp949',index_col=0)

In [3]:
# category_code_1 NaNa 값 존재 : 61317
df_top10['category_code_1'].value_counts()

electronics     1880787
appliances       286249
computers        247549
apparel          100234
furniture         71950
auto              33043
construction      28378
kids              25549
accessories       13628
sport              8863
medicine            626
stationery          242
country_yard        219
Name: category_code_1, dtype: int64

In [4]:
# category_code_2 NaNa 값 존재 : 61317
df_top10['category_code_2'].value_counts()

kitchen        215119
audio          136741
video          123861
environment     59946
living_room     35771
accessories     33043
tools           27402
peripherals     27282
components      26824
shoes           20125
bedroom         20024
personal         7657
camera           4965
bathroom         1874
fmcg             1350
universal         469
furniture          46
Name: category_code_2, dtype: int64

In [5]:
# category_code_2 NaNa 값 존재 : 61317
df_top10['category_code_3'].value_counts()

smartphone     1453828
notebook        157876
clocks          125609
tv              122874
headphone       113014
                ...   
hammok              46
shorts              31
espadrilles         21
screw                6
fryer                6
Name: category_code_3, Length: 125, dtype: int64

## Data NaN value check

In [6]:
def NaN_check(series):
    return series.isnull().sum()

In [9]:
NaN_check(df_top10['event_time']),NaN_check(df_top10['event_type'])

(0, 0)

In [10]:
NaN_check(df_top10['product_id']),NaN_check(df_top10['category_id'])

(0, 0)

In [12]:
NaN_check(df_top10['price']),NaN_check(df_top10['category_code']),NaN_check(df_top10['brand']),

(0, 813059, 347605)

## NaN 값 처리

- 데이터에서 nan 값은 category_code, brand에 존재.
- 데이터에서 nan 값을 drop하는 방법도 고안했지만 user의 유의미한 행동 패턴을 분석하고자 가급적 nan값이 포함된 행을 살리는 방향으로 선택.
- 데이터 nan 값은 총 3가지 경우

||   경우의 수   |해결방안|
|:-------|:---------|----:|
|1|category_code만 nan 값 존재| 해당 행들의 brand 중 최다 빈도로 나온 category_code로 대입|
|2|brand만 nan 값 존재| 해당 행들의 category_code 중 최다 빈도로 나온 brand로 대입 |
|3|둘 다 nan 값 존재     | 둘 다 etc로 대입 |
|4|category_code만 nan 값이지만 해당 brand에 해당하는 모든 category_code의 값들이 nan값   |   해당 brand 값 그대로 대입 |

In [13]:
# TR : True & False

df_top10['TF_category_code'] = df_top10['category_code'].isnull()
df_top10['TF_brand'] = df_top10['brand'].isnull()

In [14]:
lst_nan = df_top10.query('TF_category_code == True').query('TF_brand == True').index.to_list()
lst_category_nan = df_top10.query('TF_category_code == True').query('TF_brand == False').index.to_list()
lst_brand_nan = df_top10.query('TF_category_code == False').query('TF_brand == True').index.to_list()

In [17]:
df_top10.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,category_code_1,category_code_2,category_code_3,category_code_4,user_top10,user_session_c
0,2019-11-01 00:00:19 UTC,view,26100002,2053013553887052089,NaN,sulu,51.48,520956879,88e782d0-cb8f-4b25-b09c-6dd9bce06832,NaN,NaN,NaN,NaN,1,3
1,2019-11-01 00:00:35 UTC,view,2701305,2053013563911439225,appliances.kitchen.refrigerators,samsung,499.94,524356542,153f9818-4d32-4e8b-ba9f-f355094e8ae4,appliances,kitchen,refrigerators,NaN,1,1
2,2019-11-01 00:01:46 UTC,view,39700019,2084962304016777941,apparel.costume,NaN,136.43,516426931,b36a7084-888d-4e37-bad4-cec9cfb1fbce,apparel,NaN,costume,NaN,1,4
3,2019-11-01 00:02:02 UTC,view,1004873,2053013555631882655,electronics.smartphone,samsung,362.29,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10,electronics,NaN,smartphone,NaN,1,24
4,2019-11-01 00:02:36 UTC,view,5100816,2053013553375346967,NaN,xiaomi,29.58,524356542,2f44a6ff-59aa-458c-acef-aee764d2ce10,NaN,NaN,NaN,NaN,1,24


In [19]:
for idx in tqdm(lst_nan):
    df_top10['category_code'][idx] = 'etc'
    df_top10['brand'][idx] = 'etc'    

100%|████████████████████████████████████| 232028/232028 [20:50<00:00, 185.50it/s]


In [21]:
for idx in tqdm(lst_category_nan):
    try:
        df_top10['category_code'][idx] = df_top10.query('brand == "{}"'.format(df_top10['brand'][idx])['category_code'].value_counts().index[0])
                                                        
    except:
        df_top10['category_code'][idx] = df_top10['brand'][idx]

100%|████████████████████████████████████| 581031/581031 [25:18<00:00, 382.64it/s]


In [ ]:
df_top10.query(category_code == df_top10['category_code'][idx]

In [25]:
df_category = df_top10[['category_code','brand']]

In [28]:
for idx in tqdm(lst_brand_nan):
    category = df_category['category_code'][idx]
    brand = df_category.query(f'category_code == "{category}"')['brand'].value_counts().index[0]
    df_category['brand'][idx] = brand

100%|██████████████████████████████████| 115577/115577 [21:01:51<00:00,  1.53it/s]


In [31]:
df_top10['brand'] = df_category['brand']

In [32]:
df_top10['event_time'] = df_top10['event_time'].apply(lambda x:x[:19])

In [33]:
df_top10['weekday'] =0

In [34]:
df_top10['weekday'] = df_top10['event_time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime("%A"))

In [35]:
df_top10['weekday']

0            Friday
1            Friday
2            Friday
3            Friday
4            Friday
             ...   
3510371    Saturday
3510372    Saturday
3510373    Saturday
3510374    Saturday
3510375    Saturday
Name: weekday, Length: 3510376, dtype: object

In [36]:
df_top10['separate_time'] = 0

In [37]:
df_top10['time'] = ''

In [41]:
lst_time = df_top10['event_time'].apply(lambda x: x[11:13]).to_list()

In [43]:
time_dic = {'00':'0~3','01':'0~3','02':'0~3',
           '03':'3~6','04':'3~6','05':'3~6',
           '06':'6~9','07':'6~9','08':'6~9',
           '09':'9~12','10':'9~12','11':'9~12',
           '12':'12~15','13':'12~15','14':'12~15',
           '15':'15~18','16':'15~18','17':'15~18',
           '18':'18~21','19':'18~21','20':'18~21',
           '21':'21~24','22':'21~24','23':'21~24'}

In [44]:
for idx,time in tqdm(enumerate(lst_time)):
    df_top10['time'][idx] = time_dic[f'{time}']

3510376it [2:31:02, 387.36it/s]


## event_type 구별

- 1. view
- 2. cart
- 3. purchase

In [46]:
df_top10['user_id'] = df_top10['user_id'].apply(lambda x: str(x))

In [49]:
df_purchase = df_top10.query('event_type == "purchase"').reset_index()
df_view = df_top10.query('event_type == "view"').reset_index()
df_cart = df_top10.query('event_type == "cart"').reset_index()

del df_purchase['index']
del df_view['index']
del df_cart['index']

In [50]:
df_purchase.shape,df_cart.shape,df_view.shape

((283445, 18), (476078, 18), (2750853, 18))

In [51]:
lst_p_user_id = df_purchase['user_id'].value_counts().index.to_list()
lst_v_user_id = df_view['user_id'].value_counts().index.to_list()
lst_c_user_id = df_cart['user_id'].value_counts().index.to_list()

In [52]:
dic_p_by_user_id = {}
dic_v_by_user_id = {}
dic_c_by_user_id = {}

In [53]:
for ind in lst_p_user_id:
    dic_p_by_user_id[f'{ind}'] = 0
for ind in lst_v_user_id:
    dic_v_by_user_id['{}'.format(ind)] = 0
for ind in lst_c_user_id:
    dic_c_by_user_id['{}'.format(ind)] = 0

In [54]:
for ind in tqdm(range(len(df_purchase))):
    dic_p_by_user_id['{}'.format(df_purchase['user_id'][ind])] += df_purchase['price'][ind]
    
for ind in tqdm(range(len(df_view))):
    dic_v_by_user_id['{}'.format(df_view['user_id'][ind])] += df_view['price'][ind]
    
for ind in tqdm(range(len(df_cart))):
    dic_c_by_user_id['{}'.format(df_cart['user_id'][ind])] += df_cart['price'][ind]

100%|█████████████████████████████████| 476078/476078 [00:03<00:00, 150938.79it/s]


In [55]:
# df_user_total_price
df_user_total_price = pd.DataFrame()

In [56]:
df_user_total_price['user_id'] = dic_p_by_user_id.keys()
df_user_total_price['total_price'] = dic_p_by_user_id.values()

In [57]:
df_user_total_price = df_user_total_price.sort_values(by='total_price',ascending=False)

In [58]:
df_user_total_price.head()

,user_id,total_price
3,518514099,203986.07
1,512386086,161682.49
0,564068124,131929.20
78,515715331,117130.12
24,521230795,111072.35


In [59]:
# 상위 10% 상품 제품 누적 금액 유저 수 : 44039

df_user_total_price['user_id'].nunique()

44163

In [60]:
# 10그룹 나누기 위한 숫자

n = int(len(df_user_total_price)*0.1)

In [61]:
dic_group = {}
for ind in tqdm(range(n*9,len(df_user_total_price))):
    dic_group['{}'.format(df_user_total_price['user_id'][ind])] = 10

100%|█████████████████████████████████████| 4419/4419 [00:00<00:00, 134611.30it/s]


In [62]:
df_purchase['group'] = 0

In [63]:
df_purchase['group'].value_counts()

0    283445
Name: group, dtype: int64

## per user_id dataframe

In [208]:
list_user_id = df_top10['user_id'].unique().tolist()
df_user_id = pd.DataFrame()
df_user_id['user_id'] = list_user_id

In [210]:
# event_type == view
df_view = df_top10.query('event_type == "view"')

f = df_view.groupby('user_id').agg({
    'event_type' : [('view', lambda x: x.count())],
    'brand' : [('v_brand', lambda x: x.nunique())],
    'price' : [('view_price_avg', lambda x: np.mean(x))]
})

f.columns = f.columns.droplevel()  # 동일한 코드: f.columns = [j for _, j in f.columns]
f=f.reset_index()

lst_user_id = df_user_id['user_id'].to_list()
lst_f = f['user_id'].to_list()
lst = []
compare_lst = [0]*len(f.columns)
for idx,user_id in tqdm(enumerate(lst_user_id)):
    if user_id not in lst_f:
        lst.append(user_id)
for idx,user_id in enumerate(lst, start=len(f)):
    compare_lst[0] = user_id
    f.loc[idx] = compare_lst
df_user_id = df_user_id.merge(f, how='left')

44163it [00:26, 1687.22it/s]


In [ ]:
f = df_purchase.groupby('user_id')['event_type'].agg({
    'event_type' : [('purchase', lambda x: x.count())],
}).reset_index()
df_user_id = df_user_id.merge(f, how='left')

In [209]:
# event_type == purchase

df_purchase = df_top10.query('event_type == "purchase"')

f = df_purchase.groupby('user_id').agg({
    'event_type' : [('purchase', lambda x: x.count())],
    'brand' : [('p_brand', lambda x: x.nunique())],
    'price' : [('view_price_avg' , lambda x: np.mean(x)),
               ('t_purchase', lambda x: np.sum(x)),
               ('purchase_price_max', lambda x: np.max(x)),
               ('purchase_price_min', lambda x: np.min(x))]
})

f.columns = f.columns.droplevel()
f=f.reset_index()

lst_user_id = df_user_id['user_id'].to_list()
lst_f = f['user_id'].to_list()
lst = []
compare_lst = [0]*len(f.columns)
for idx,user_id in tqdm(enumerate(lst_user_id)):
    if user_id not in lst_f:
        lst.append(user_id)
for idx,user_id in enumerate(lst, start=len(f)):
    compare_lst[0] = user_id
    f.loc[idx] = compare_lst
df_user_id = df_user_id.merge(f, how='left')

44163it [00:23, 1877.42it/s]


In [211]:
# event_type == cart

df_cart = df_top10.query('event_type == "cart"')

f = df_cart.query('event_type == "cart"').groupby('user_id').agg({
    'event_type' : [('cart', lambda x: x.count())],
    'brand' : [('c_brand', lambda x: x.count())],
    'price' : [('cart_price_avg' , lambda x: np.mean(x))]
})

f.columns = f.columns.droplevel()
f=f.reset_index()
lst_user_id = df_user_id['user_id'].to_list()
lst_f = f['user_id'].to_list()
lst = []
compare_lst = [0]*len(f.columns)
for idx,user_id in tqdm(enumerate(lst_user_id)):
    if user_id not in lst_f:
        lst.append(user_id)
for idx,user_id in enumerate(lst, start=len(f)):
    compare_lst[0] = user_id
    f.loc[idx] = compare_lst
df_user_id = df_user_id.merge(f, how='left')

44163it [00:24, 1835.08it/s]


In [212]:
df_user_id

,user_id,purchase,p_brand,view_price_avg,t_purchase,purchase_price_max,purchase_price_min,view,v_brand,cart,c_brand,cart_price_avg
0,520956879,12,4,253.684167,3044.21,952.15,45.56,NaN,NaN,17,17,230.599412
1,524356542,5,1,296.552000,1482.76,375.30,269.49,NaN,NaN,6,6,279.213333
2,516426931,11,4,303.706364,3340.77,488.80,53.54,NaN,NaN,10,10,353.712000
3,554673298,4,2,444.735000,1778.94,1585.11,64.61,NaN,NaN,8,8,1466.363750
4,562210838,1,1,1583.480000,1583.48,1583.48,1583.48,NaN,NaN,1,1,1583.480000
...,...,...,...,...,...,...,...,...,...,...,...,...
44158,579940508,2,2,821.630000,1643.26,849.16,794.10,NaN,NaN,1,1,794.100000
44159,579953265,1,1,1544.160000,1544.16,1544.16,1544.16,NaN,NaN,3,3,1286.740000
44160,515542288,1,1,1634.380000,1634.38,1634.38,1634.38,2.0,1.0,1,1,1634.380000
44161,579961103,1,1,1634.380000,1634.38,1634.38,1634.38,2.0,1.0,1,1,1634.380000


In [199]:
# 전체 데이터

f = df_top10.groupby('user_id').agg({
    'category_code' : [('c_category_code', lambda x: x.nunique()),
                       ('category_code_max', lambda x: x.value_counts().index[0]),
                       ('category_code_min', lambda x: x.value_counts().index[-1])],
    'user_session' : [('c_user_session', lambda x: x.count()),
                      ('number_behavior', lambda x: x.nunique())],
    'product_id' : [('상품종류', lambda x: x.nunique())],
    'weekday' : [('weekday_max', lambda x: max(x))],
    'time' : [('time_max', lambda x: max(x))]
})
f.columns = f.columns.droplevel()  # 동일한 코드: f.columns = [j for _, j in f.columns]
f=f.reset_index()
df_user_id = df_user_id.merge(f, how='left')

df_user_id['CV'] = df_user_id['cart']/df_user_id['view']
df_user_id['PC'] = df_user_id['purchase']/df_user_id['cart']
df_user_id['PV'] = df_user_id['purchase']/df_user_id['view']

df_user_id['VS'] = df_user_id['view']/df_user_id['number_behavior']
df_user_id['CS'] = df_user_id['cart']/df_user_id['number_behavior']
df_user_id['PS'] = df_user_id['purchase']/df_user_id['number_behavior']

In [200]:
df_user_id

,user_id,view,v_brand,view_price_avg,purchase,p_brand,t_purchase,purchase_price_max,purchase_price_min,cart,c_brand,cart_price_avg,c_category_code,category_code_max,category_code_min,c_user_session,number_behavior,상품종류,weekday_max,time_max,CV,PC,PV,VS,CS,PS
0,520956879,98,17,269.477755,NaN,NaN,NaN,NaN,NaN,17,17,230.599412,17,electronics.audio.headphone,tefal,127,37,46,Wednesday,9~12,0.173469,NaN,NaN,2.648649,0.459459,NaN
1,524356542,187,18,275.975615,NaN,NaN,NaN,NaN,NaN,6,6,279.213333,17,electronics.smartphone,lucente,198,24,85,Wednesday,9~12,0.032086,NaN,NaN,7.791667,0.250000,NaN
2,516426931,59,11,344.211186,NaN,NaN,NaN,NaN,NaN,10,10,353.712000,10,sony,xiaomi,80,13,27,Saturday,3~6,0.169492,NaN,NaN,4.538462,0.769231,NaN
3,554673298,78,6,1083.816923,NaN,NaN,NaN,NaN,NaN,8,8,1466.363750,6,electronics.smartphone,nokian,90,21,24,Wednesday,9~12,0.102564,NaN,NaN,3.714286,0.380952,NaN
4,562210838,11,1,1527.590000,NaN,NaN,NaN,NaN,NaN,1,1,1583.480000,1,electronics.smartphone,electronics.smartphone,13,1,6,Friday,0~3,0.090909,NaN,NaN,11.000000,1.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44158,579940508,3,2,830.806667,NaN,NaN,NaN,NaN,NaN,1,1,794.100000,1,electronics.smartphone,electronics.smartphone,6,1,2,Saturday,21~24,0.333333,NaN,NaN,3.000000,1.000000,NaN
44159,579953265,5,1,1235.256000,NaN,NaN,NaN,NaN,NaN,3,3,1286.740000,1,electronics.smartphone,electronics.smartphone,9,1,2,Saturday,21~24,0.600000,NaN,NaN,5.000000,3.000000,NaN
44160,515542288,2,1,1634.380000,1.0,1.0,1634.38,1634.38,1634.38,1,1,1634.380000,1,electronics.smartphone,electronics.smartphone,4,1,1,Saturday,21~24,0.500000,1.0,0.5,2.000000,1.000000,1.0
44161,579961103,2,1,1634.380000,1.0,1.0,1634.38,1634.38,1634.38,1,1,1634.380000,1,electronics.smartphone,electronics.smartphone,4,1,1,Saturday,21~24,0.500000,1.0,0.5,2.000000,1.000000,1.0


In [169]:
from scipy.stats import norm

x = df_top10['price'].mean()
sd = df_top10['price'].std()
cnt = df_top10['price'].count()
value = norm.ppf(0.9, x, sd)
f = df_top10[df_top10['price'] > value].groupby('user_id')['price'].agg([
    ('고가상품구매건수', np.size)
]).reset_index()
df_user_id = df_user_id.merge(f, how='left')
df_user_id
# df_user_id['고가상품구매율'] = df_user_id['고가상품구매건수'] / df_user_id['구매건수'].astype(float)
# features

,user_id,view,v_brand,view_price_avg,purchase,p_brand,t_purchase,purchase_price_max,purchase_price_min,cart,c_brand,cart_price_avg,c_category_code,category_code_max,category_code_min,c_user_session,number_behavior,상품종류,weekday_max,time_max,CV,PC,PV,VS,CS,PS,고가상품구매건수
0,520956879,98.0,17.0,269.477755,NaN,NaN,NaN,NaN,NaN,17.0,17.0,230.599412,17,electronics.audio.headphone,tefal,127,37,46,Wednesday,9~12,0.173469,NaN,NaN,2.648649,0.459459,NaN,3.0
1,524356542,187.0,18.0,275.975615,NaN,NaN,NaN,NaN,NaN,6.0,6.0,279.213333,17,electronics.smartphone,lucente,198,24,85,Wednesday,9~12,0.032086,NaN,NaN,7.791667,0.250000,NaN,NaN
2,516426931,59.0,11.0,344.211186,NaN,NaN,NaN,NaN,NaN,10.0,10.0,353.712000,10,sony,xiaomi,80,13,27,Saturday,3~6,0.169492,NaN,NaN,4.538462,0.769231,NaN,NaN
3,554673298,78.0,6.0,1083.816923,NaN,NaN,NaN,NaN,NaN,8.0,8.0,1466.363750,6,electronics.smartphone,nokian,90,21,24,Wednesday,9~12,0.102564,NaN,NaN,3.714286,0.380952,NaN,54.0
4,562210838,11.0,1.0,1527.590000,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1583.480000,1,electronics.smartphone,electronics.smartphone,13,1,6,Friday,0~3,0.090909,NaN,NaN,11.000000,1.000000,NaN,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44158,579940508,3.0,2.0,830.806667,NaN,NaN,NaN,NaN,NaN,1.0,1.0,794.100000,1,electronics.smartphone,electronics.smartphone,6,1,2,Saturday,21~24,0.333333,NaN,NaN,3.000000,1.000000,NaN,NaN
44159,579953265,5.0,1.0,1235.256000,NaN,NaN,NaN,NaN,NaN,3.0,3.0,1286.740000,1,electronics.smartphone,electronics.smartphone,9,1,2,Saturday,21~24,0.600000,NaN,NaN,5.000000,3.000000,NaN,9.0
44160,515542288,2.0,1.0,1634.380000,1.0,1.0,1634.38,1634.38,1634.38,1.0,1.0,1634.380000,1,electronics.smartphone,electronics.smartphone,4,1,1,Saturday,21~24,0.500000,1.0,0.5,2.000000,1.000000,1.0,4.0
44161,579961103,2.0,1.0,1634.380000,1.0,1.0,1634.38,1634.38,1634.38,1.0,1.0,1634.380000,1,electronics.smartphone,electronics.smartphone,4,1,1,Saturday,21~24,0.500000,1.0,0.5,2.000000,1.000000,1.0,4.0
